# This notebook is used to fit QRM intensities from EURONEXT data
# Imports

In [ ]:
# Imports
from IPython.display import display, HTML
import os
import h5py
import pandas as pd, datetime as dt, numpy as np, matplotlib.pyplot as plt
import seaborn as sns
from pandas.tseries.offsets import DateOffset
import sys
import datetime as dt
from pathlib import Path

# Display options
thisnotebooksys = sys.stdout
pd.set_option('display.width', 1000)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('mode.chained_assignment', None)

#  Data Description

- DTIME : le timestamp de l'ordre
- ORDER_ID : l'identifiant de l'ordre
- PRICE 
- QTY
- ORDER_SIDE
- ORDER_SIDE
- ORDER_TYPE :  <br>1 pour Market Order; <br>2 pour Limit Order; <br>q pour Quote <br> W pour Market On Open;
- ACTION_TYPE : <br> I = limit order insertion (passive); <br> C = limit order cacnellations; <br> R = replace order that lose priority; <br> r = replace order that keeps priority; <br> S = replace order that makes the order aggressive (give rise to trade); <br> T = aggressive order (give rise to trade)
- MATCH_STRATEGY : True/False
- IS_OPEN_TRADE :  True/False

## Get Data and clean it

In [ ]:
filename= 'day20160428'
df = pd.read_hdf(Path(r'..\data', f'{filename}.h5'))

# Clean Data
df = df[((df.DTIME.dt.time < dt.time(21)) & (df.DTIME.dt.time > dt.time(6)))] #no auction

df = df[df.ASK0!=0]
df = df[df.BID0!=0]
df = df[((df.PRICE < 5000) & (df.PRICE > 4000))]
df = df[((df.ASK0 < 5000) & (df.ASK0 > 4000))]
df = df[((df.BID0 < 5000) & (df.BID0 > 4000))]

# get the first future
df = df[df.PRODUCT_NAME=='JFFCE160500000F']

# ticksize
ticksize = 0.5

#Construct columns for better vizu
newcols = ['DTIME', 'ORDER_ID', 'PRICE', 'QTY', 
           'ORDER_SIDE', 'ORDER_TYPE', 'ACTION_TYPE'] 

# df.loc[:, newcols+newcols2]

In [ ]:
# nb of queues for fitting
nbQueues = 4

BIDASK_COLS = []
BIDASK_COLS2 = []
for i in range(nbQueues-1, -1, -1):
    BIDASK_COLS += [f'BID{i}']
    BIDASK_COLS2 += [f'BID{i}', f'BID{i}_QTY']
for i in range(nbQueues):
    BIDASK_COLS += [f'ASK{i}']
    BIDASK_COLS2 += [f'ASK{i}', f'ASK{i}_QTY']
    
df = df.loc[:, ['DTIME', 'ORDER_ID', 'PRICE', 'QTY', 'ORDER_SIDE', 'ORDER_TYPE', 'ACTION_TYPE'] + BIDASK_COLS2]

In [ ]:
# df

## Define events : type & queue of each event 
#### Queue on which each event is considered on

In [ ]:
%%time

# specify the limit of the event
AqueuePricesCols = [col for col in df.columns if (('ASK' in col) & ('QTY' not in col))]
BqueuePricesCols = [col for col in df.columns if (('BID' in col) & ('QTY' not in col))]

# get the queue of the event
def getQueue(x):
    if x['ORDER_SIDE'] == 'B':
        return (x['PRICE']-x[BqueuePricesCols]).abs().astype(float).idxmin() if 0 in (x['PRICE']-x[BqueuePricesCols]).values else np.nan
    else:
        return (x['PRICE']-x[AqueuePricesCols]).abs().astype(float).idxmin() if 0 in (x['PRICE']-x[AqueuePricesCols]).values else np.nan

df['eventQueue'] = df.apply(getQueue, axis=1)

In [ ]:
# Filter on queues that we want
df = df[df.eventQueue.isin(BIDASK_COLS)]

#### Event type

In [ ]:
%%time

# for replace orders, define previous queue and previous quantity
df['PREV_ORDER_PRICE'] = 0
df['PREV_ORDER_QTY']   = 0
df['PREV_ORDER_QUEUE'] = 0

# only orders with a modification that loses priority
listorders = df[df.ACTION_TYPE=='R'].ORDER_ID.unique()

#'PREV_ORDER_QUEUE'
for order_id in listorders:
    df.loc[df[df.ORDER_ID==order_id].index, ['PREV_ORDER_PRICE', 'PREV_ORDER_QTY']] = df.loc[df[df.ORDER_ID==order_id].index, ['PRICE', 'QTY']].shift(1).values

In [ ]:
%%time

# define prev order queue
def getPrevQueue(x):
    if x['ORDER_SIDE'] == 'B':
        return (x['PREV_ORDER_PRICE']-x[BqueuePricesCols]).abs().astype(float).idxmin() if 0 in (x['PREV_ORDER_PRICE']-x[BqueuePricesCols]).values else np.nan
    else:
        return (x['PREV_ORDER_PRICE']-x[AqueuePricesCols]).abs().astype(float).idxmin() if 0 in (x['PREV_ORDER_PRICE']-x[AqueuePricesCols]).values else np.nan

df.loc[df[df.ACTION_TYPE=='R'].index, 'PREV_ORDER_QUEUE'] = df[df.ACTION_TYPE=='R'].apply(getPrevQueue, axis=1).values

In [ ]:
%%time
# define type of event (0 for removal and 1 for addition of liquidity)
# first, removal of liquidity
df.loc[df[df.ORDER_TYPE==1].index, 'eventType'] = 0
df.loc[df[((df.ORDER_TYPE=='2') & (df.ACTION_TYPE=='I'))].index, 'eventType'] = 1 # limit order
df.loc[df[((df.ORDER_TYPE=='2') & (df.ACTION_TYPE=='C'))].index, 'eventType'] = 0 # cancelation 
df.loc[df[((df.ORDER_TYPE=='2') & (df.ACTION_TYPE=='r'))].index, 'eventType'] = 0 # replace with quantity diminition
df.loc[df[((df.ORDER_TYPE=='2') & (df.ACTION_TYPE=='S'))].index, 'eventType'] = 0 # market order
df.loc[df[((df.ORDER_TYPE=='2') & (df.ACTION_TYPE=='T'))].index, 'eventType'] = 0 # market order

# duplicate replace orders with actgion type == R
# they are considered a liquidity removal on their previous queue
# they are considered a liquidity addition on their current queue
newdf = df[df.ACTION_TYPE=='R'].copy()
newdf['eventQueue'] = df['PREV_ORDER_QUEUE']
newdf['eventType'] = 0 # liquidity removal on previous queue
df.loc[df[((df.ORDER_TYPE=='2') & (df.ACTION_TYPE=='R'))].index, 'eventType'] = 1 # liquidity addition on current queue
df = pd.concat([df, newdf]).sort_index().reset_index(drop=True)

#### Waiting Times between events on the same queue

In [ ]:
%%time

df['WAITING_TIME'] = 0
for queue in BIDASK_COLS:
    df.loc[df[df.eventQueue==queue].index, 'WAITING_TIME'] = df.loc[df[df.eventQueue==queue].index, 'DTIME'].diff(1).apply(lambda x: x.total_seconds()).fillna(0).values

## REF PRICE

In [ ]:
%%time

# nb of ticks between best limits
df['nbticksBetweenBestLimits'] = (df['ASK0'] - df['BID0'])/ticksize

# define ref price
df_midprice = (df['BID0'] + df['ASK0'])/2
df_mid1 = df_midprice + ticksize/2
df_mid2 = df_midprice - ticksize/2
df_refprice_odd = df_midprice * (df['nbticksBetweenBestLimits']%2)

# define ref price
df['refprice'] = df_refprice_odd

lastprice = df_midprice.iloc[0]
for (i, idx) in enumerate(list(df.index)):
    # odd values
    if df.at[idx, 'refprice'] == 0:
        df.at[idx, 'refprice'] = df_mid1.at[idx] if np.abs(lastprice-df_mid1.at[idx]) < np.abs(lastprice-df_mid2.at[idx]) else df_mid2.at[idx]
    lastprice = df.at[idx, 'refprice']
#     if i%100000==1:
#         print(i)

## AES

In [ ]:
totalcountEvents2 = df['eventQueue'].value_counts()
totalcountEvents2.loc[BIDASK_COLS]
SizesOfEvents = df[['eventQueue', 'QTY']].groupby('eventQueue').sum()
dfAES = SizesOfEvents
dfAES.columns = ['sizes']
dfAES.loc[BIDASK_COLS, 'counts'] = totalcountEvents2.loc[BIDASK_COLS]
dfAES = dfAES.loc[BIDASK_COLS]
AES = (dfAES['sizes'] / dfAES['counts'])

## Lambdas

In [ ]:
%%time 

# construct new df with data that we want : event type, event time, event queue, event's waiting time, every queue's size
df_lambdas = df.loc[:, ['DTIME', 'eventType', 'eventQueue', 'WAITING_TIME'] + [f'{col}_QTY' for col in BIDASK_COLS]]
df_lambdas = df_lambdas[df_lambdas.eventQueue.isin(BIDASK_COLS)]
df_lambdas.columns = ['DTIME', 'eventType', 'eventQueue', 'WAITING_TIME'] + BIDASK_COLS

# devide sizes by corresponding AES's
for col in BIDASK_COLS:
    df_lambdas[col] = (df_lambdas[col] / AES[col]).apply(np.ceil) #rounded up

In [ ]:
df_lambdas.to_hdf(f'df_lambdas_{filename}.h5', 'tmp')

In [ ]:
df_lambdas = pd.concat([pd.read_hdf(f'df_lambdas_{filename}.h5') for filename in ['day20160425', 'day20160426', 'day20160427', 'day20160428', 'day20160429']])
len(df_lambdas)

In [ ]:
# Get lambda for all queues, for each state of the queue
Lambda_plus = pd.DataFrame({'queue' : BIDASK_COLS}).set_index('queue').T
Lambda_minus = pd.DataFrame({'queue' : BIDASK_COLS}).set_index('queue').T

allQuantities = np.sort(pd.unique(df_lambdas.loc[:, BIDASK_COLS].values.ravel()))

# until 40
allQuantities = [qtty for qtty in allQuantities if qtty<=40]

ALPHAs = df_lambdas.groupby('eventQueue').WAITING_TIME.mean() #df_lambdas[df_lambdas.eventQueue==curr_queue].WAITING_TIME.mean()
lendf = len(df)

In [ ]:
%%time

for qtty in allQuantities:
    Lambda_plus.loc[qtty] = np.nan
    Lambda_minus.loc[qtty] = np.nan

    for curr_queue in BIDASK_COLS:
        ALPHA = ALPHAs[curr_queue]
        tmpdf = df_lambdas[((df_lambdas.eventQueue==curr_queue) & (df_lambdas[curr_queue]==qtty))]
        
        len1 = len(tmpdf[tmpdf.eventType==1])
        len0 = len(tmpdf[tmpdf.eventType==0])
        len2 = len(tmpdf)
        
        if len2 == 0:
            Lambda_plus.at[qtty, curr_queue] = np.nan
            Lambda_minus.at[qtty, curr_queue] = np.nan
        else:
            Lambda_plus.at[qtty, curr_queue] = (len1/len2) / ALPHA
            Lambda_minus.at[qtty, curr_queue] = (len0/len2) / ALPHA

In [ ]:
(Lambda_plus.fillna(0).BID0/Lambda_minus.fillna(0).BID0).plot()

## Visualize some statistics

In [ ]:
# plot
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8), (ax9, ax10), (ax11, ax12)) = plt.subplots(6,2,figsize=(25,40), sharex=False)

# Ref prices distribution
countRefPrices = df.groupby('refprice')['refprice'].count()
ax1.bar(countRefPrices.index, countRefPrices.values); ax1.legend(['reference price distribution'])

# Ref Price evolution
df.refprice.plot(ax=ax2); ax2.legend(['Ref Price'])

# AES distribtion
ax3.bar(AES.index, AES.values); ax3.legend(['Average Event Size per Queue']); 

# Waiting times distribution
WT = df[['WAITING_TIME', 'eventQueue']].groupby('eventQueue').mean().loc[BIDASK_COLS]
ax4.bar(WT.index, WT.values[:,0]); ax4.legend(['Average Waiting Time']); 

# evolution of intensities depending on AES for best limits
Lambda_plus.loc[:, 'BID0'].fillna(0).plot(ax=ax5, linestyle='-.', marker='o'); ax5.legend(['Liquidity Addition - First Bid Limit']); 
Lambda_plus.loc[:, 'ASK0'].fillna(0).plot(ax=ax6, linestyle='-.', marker='o'); ax6.legend(['Liquidity Addition - First Ask Limit']); 

Lambda_minus.loc[:, 'BID0'].fillna(0).plot(ax=ax7, linestyle='-.', marker='o'); ax7.legend(['Liquidity Removal - First Ask Limit']); 
Lambda_minus.loc[:, 'ASK0'].fillna(0).plot(ax=ax8, linestyle='-.', marker='o'); ax8.legend(['Liquidity Removal - First Bid Limit']); 

# evolution of intensities depending on AES for second limits
Lambda_plus.loc[:, 'BID1'].fillna(0).plot(ax=ax9, linestyle='-.', marker='o'); ax9.legend(['Liquidity Addition - Second Bid Limit']); 
Lambda_plus.loc[:, 'ASK1'].fillna(0).plot(ax=ax10, linestyle='-.', marker='o'); ax10.legend(['Liquidity Addition - Second Ask Limit']); 

Lambda_minus.loc[:, 'BID1'].fillna(0).plot(ax=ax11, linestyle='-.', marker='o'); ax11.legend(['Liquidity Removal - Second Bid Limit']); 
Lambda_minus.loc[:, 'ASK1'].fillna(0).plot(ax=ax12, linestyle='-.', marker='o'); ax12.legend(['Liquidity Removal - Second Ask Limit']); 

fig.autofmt_xdate()

# Pickelize results

In [ ]:
Lambda_plus.T.to_pickle(r'..\data\Lambda_plus.pkl')
Lambda_minus.T.to_pickle(r'..\data\Lambda_minus.pkl')
AES.to_pickle(r'..\data\event_sizes.pkl')

In [ ]:
# AES